# ENTREGABLE DATA SCIENCE PARA FINANZAS
### Alfonso Calvete, Federica Cueto, Lucía Otero
### 15 de diciembre de 2023

En este proyecto se propone generar una base de datos mensual de los precios de los ejemplares vendidos en la sección ficción de la tienda bookshop para analizar su evolución.
Para esto, se hace webscraping de la página https://www.bookshop.com.uy de distintas categorías dentro de la sección ficción (cambiando la sección se podría recoger información de otros ejemplares). 
Se guarda la información presente en la página como un dataframe que luego se exporta como csv.


## Setup
Se instalan las librerías

In [34]:
!pip install pandas
!pip install numpy
!pip install beautifulsoup4

## Webscraping

Se obtiene de la cantidad de páginas de las categorías elegidas de www.bookshop.com.uy

In [38]:
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup

# Se define User Agent para evitar errores 403
ua = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
          'AppleWebKit/537.36 (KHTML, like Gecko) '
          'Chrome/51.0.2704.103 Safari/537.36'}
# URL
rootURL = "https://www.bookshop.com.uy"
urls_bookshop = ["/ficcion/clasicos.html",
                 "/ficcion/narrativa-nacional.html",
                 "/ficcion/novela-policial.html", 
                 "/ficcion/libros-ilustrados.html"
                ]
total_pages = []

# Se extrae el número de páginas a scrapear por categoría
for url in urls_bookshop:
  page_number = 1
  last_safe_page = 0
  safety = False
  while(True):
    response = requests.get(rootURL + url + "?p=" + str(page_number), headers=ua)
    soup = BeautifulSoup(response.text, 'html.parser')
    current_page = soup.find("li", {"class": "item current"})
    current_page_number =  current_page.find("span", {"class": None}).text
    if(current_page_number == str(page_number)):
      last_safe_page = page_number
      if safety :
        page_number += 1
      else:
        page_number += 5 
    else:
      if(page_number == last_safe_page + 1):
        print("URL " + url + " has " + str(last_safe_page) + " pages")
        total_pages.append(last_safe_page)
        break
      page_number = last_safe_page + 1

URL /ficcion/clasicos.html has 11 pages
URL /ficcion/narrativa-nacional.html has 8 pages
URL /ficcion/novela-policial.html has 30 pages
URL /ficcion/libros-ilustrados.html has 2 pages


Se realiza el scraping 

In [39]:
links = []
for (i,url) in enumerate(urls_bookshop):
    pag = list(range(1, total_pages[i]+1))
    print(f"Scraping category {url}, a total of {total_pages[i]} to scrap")
    for i in pag:
        url1 = f"{rootURL}{url}?p={i}"
        response = requests.get(url1, headers=ua)
        soup = BeautifulSoup(response.text, 'html.parser')
        links1 = [a['href'] for a in soup.find_all('a', class_='product-item-link')]
        links.extend(links1)

Scraping category /ficcion/clasicos.html, a total of 11 to scrap
Scraping category /ficcion/narrativa-nacional.html, a total of 8 to scrap
Scraping category /ficcion/novela-policial.html, a total of 30 to scrap
Scraping category /ficcion/libros-ilustrados.html, a total of 2 to scrap


In [40]:
# Cantidad de links obtenidos
print (len(links))

2343


In [38]:
# NOTA: nodos html a extraer de cada link

# titulo data-ui-id="page-title-wrapper"
# precio data-price-type="finalPrice"
# Disponibilidad .stock.available span[classˆ="label"]
# ISBN itemprop="sku"
# autor data-th="Autor"
# editorial data-th="Editorial"
# Edicion data-th="Edición"
# Idioma data-th="Idioma"
# Páginas data-th="Páginas"
# Tipo de tapa data-th="Tipo"
# Descrpcion itemprop="description"

Se define la función para extraer la información de cada ejemplar. 
Se transforma la información a minúsculas para facilitar las búsquedas posteriores.
Se incluye la fecha para trazabilidad.

In [41]:
def parse_book_info(link, headers):
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    book_info = {}
    book_info['titulo'] = str(soup.select_one('[data-ui-id="page-title-wrapper"]').text).lower()
    book_info['link'] = link
    if (soup.select_one('[data-price-type="finalPrice"]') != None):
        book_info['precio'] = soup.select_one('[data-price-type="finalPrice"]')['data-price-amount']
    if (soup.select_one('.stock.available span:not([class="label"])') != None):
        book_info['disponibilidad'] = str(soup.select_one('.stock.available span:not([class="label"])').text).lower()
    if (soup.select_one('[itemprop="sku"]') != None):
        book_info['ISBN'] = soup.select_one('[itemprop="sku"]').text
    if (soup.select_one('[data-th="Autor"]') != None):
        book_info['autor'] = str(soup.select_one('[data-th="Autor"]').text).lower()
    if (soup.select_one('[data-th="Editorial"]') != None):
        book_info['editorial'] = str(soup.select_one('[data-th="Editorial"]').text).lower()
    if (soup.select_one('[data-th="Edición"]') != None):
        book_info['edicion'] = str(soup.select_one('[data-th="Edición"]').text).lower()
    if (soup.select_one('[data-th="Idioma"]') != None):
        book_info['idioma'] = str(soup.select_one('[data-th="Idioma"]').text).lower()
    if (soup.select_one('[data-th="Páginas"]') != None):
        book_info['paginas'] = soup.select_one('[data-th="Páginas"]').text.lower()
    if (soup.select_one('[data-th="Tipo"]') != None):
        book_info['tapa'] = str(soup.select_one('[data-th="Tipo"]').text).lower()
    if (soup.select_one('[itemprop="description"]') != None):
        book_info['descripcion'] = str(soup.select_one('[itemprop="description"]').text).lower()
    book_info['fecha'] = datetime.date.today().strftime("%Y-%m-%d")  # Agrego la fecha de scraping
    return book_info


Se extrae la información de cada ejemplar.

In [43]:
values = []
contador = 0
for link in links:
    contador += 1
    parsed = parse_book_info(link,ua)
    values.append(parsed)
    if (contador % 100 == 0):
        print(str(contador) + " - Parsed: " + str(parsed))
print(values)

100 - Parsed: {'titulo': 'el sabueso de los baskerville ', 'link': 'https://www.bookshop.com.uy/el-sabueso-de-los-baskerville.html', 'precio': '295', 'disponibilidad': 'disponible', 'ISBN': '9788417244446', 'autor': 'conan doyle, sir arthur ', 'editorial': 'edimat libros ', 'edicion': '2022', 'idioma': 'español', 'tapa': 'rústica', 'descripcion': 'sherlock holmes se ha convertido en un símbolo en el género detectivesco; es, sin lugar a dudas, el investigador por antonomasia. cuando alguien piensa en un detective lo primero que le viene a la mente es la silueta alta y angulosa de este residente de baker street, con su pipa y su curioso sombrero, tal como la plasmase el ilustrador sidney paget. la popularidad de este personaje llegó a ser tal que superó a la de su propio creador, sir arthur conan doyle. sus excepcionales y sorprendentes intuiciones son siempre el brillante resultado de un largo y exhaustivo proceso deductivo, basado en una extraordinaria capacidad de observación que deja

Se crea el dataframe

In [44]:
book_info = pd.DataFrame(values)
display(book_info.head())

,titulo,link,precio,disponibilidad,ISBN,autor,editorial,edicion,idioma,paginas,tapa,descripcion,fecha
0,pride and prejudice,https://www.bookshop.com.uy/pride-and-prejudic...,790,disponible,9781840227932,jane austen,9781840227932,2023,español,160,tapa dura,pride and prejudice by jane austen. austen's m...,2023-12-13
1,novela de ajedrez,https://www.bookshop.com.uy/novela-de-ajedrez....,890,disponible,9788418933523,stefan zweig,américa latina,2023,español,96,tapa dura,un crucero que cubre la ruta de nueva york a b...,2023-12-13
2,la llamada de cthulhu y otras historias,https://www.bookshop.com.uy/la-llamada-de-cthu...,290,disponible,9788417782283,howard phillips lovecraft,mestas,2021,español,128,rústica,la llamada de cthulhu es la obra central entor...,2023-12-13
3,sueño en el pabellón rojo ii,https://www.bookshop.com.uy/sue-o-en-el-pabell...,1350,disponible,9788481098358,cao xueqin,galaxia gutenberg,2023,español,1195,rústica,segundo volumen del gran clásico de la literat...,2023-12-13
4,orgullo y prejuicio,https://www.bookshop.com.uy/orgullo-y-prejuici...,995,disponible,9786071436092,jane austen,mirlo,2021,español,480,tapa dura,jane austen una de las autoras más reconocidas...,2023-12-13


Se convierte a csv

In [45]:
display(book_info.head(30))
print(book_info.shape)
book_info.to_csv('bookshop_1.csv', index=False)


,titulo,link,precio,disponibilidad,ISBN,autor,editorial,edicion,idioma,paginas,tapa,descripcion,fecha
0,pride and prejudice,https://www.bookshop.com.uy/pride-and-prejudic...,790,disponible,9781840227932,jane austen,9781840227932,2023,español,160,tapa dura,pride and prejudice by jane austen. austen's m...,2023-12-13
1,novela de ajedrez,https://www.bookshop.com.uy/novela-de-ajedrez....,890,disponible,9788418933523,stefan zweig,américa latina,2023,español,96,tapa dura,un crucero que cubre la ruta de nueva york a b...,2023-12-13
2,la llamada de cthulhu y otras historias,https://www.bookshop.com.uy/la-llamada-de-cthu...,290,disponible,9788417782283,howard phillips lovecraft,mestas,2021,español,128,rústica,la llamada de cthulhu es la obra central entor...,2023-12-13
3,sueño en el pabellón rojo ii,https://www.bookshop.com.uy/sue-o-en-el-pabell...,1350,disponible,9788481098358,cao xueqin,galaxia gutenberg,2023,español,1195,rústica,segundo volumen del gran clásico de la literat...,2023-12-13
4,orgullo y prejuicio,https://www.bookshop.com.uy/orgullo-y-prejuici...,995,disponible,9786071436092,jane austen,mirlo,2021,español,480,tapa dura,jane austen una de las autoras más reconocidas...,2023-12-13
5,1984 - rebelión en la granja,https://www.bookshop.com.uy/1984-rebelion-en-l...,960,disponible,9788497945592,george orwell,edimat,2023,español,352,tapa dura,"la novela 1984 es un ejemplo, quizá el más con...",2023-12-13
6,cuentos de amor de locura y de muerte,https://www.bookshop.com.uy/cuentos-de-amor-de...,530,disponible,9789877253528,horacio quiroga,de bolsillo,2023,español,160,rústica,«pasa algo muy especial con los cuentos de qui...,2023-12-13
7,sueño en el pabellón rojo,https://www.bookshop.com.uy/sue-o-en-el-pabell...,1390,disponible,9788418526800,cao xueqin,océano,2023,español,220,rústica,sueño en el pabellón rojo es el gran clásico d...,2023-12-13
8,la metamorfosis,https://www.bookshop.com.uy/la-metamorfosis-ka...,450,disponible,9788412669701,"kafka, franz",océano,gran travesia oceano,español,NaN,rústica,el inmortal relato sobre los horrores del camb...,2023-12-13
9,1984 ‐ edición ilustrada,https://www.bookshop.com.uy/1984-edicion-ilust...,1290,disponible,9788419260154,george orwell,nova,2023,español,336,rústica,NaN,2023-12-13


(2343, 13)


In [46]:
book_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2343 entries, 0 to 2342
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   titulo          2343 non-null   object
 1   link            2343 non-null   object
 2   precio          2343 non-null   object
 3   disponibilidad  1316 non-null   object
 4   ISBN            2343 non-null   object
 5   autor           2316 non-null   object
 6   editorial       2321 non-null   object
 7   edicion         2137 non-null   object
 8   idioma          2322 non-null   object
 9   paginas         2034 non-null   object
 10  tapa            2040 non-null   object
 11  descripcion     2311 non-null   object
 12  fecha           2343 non-null   object
dtypes: object(13)
memory usage: 238.1+ KB


## Búsqueda de información en el dataframe creado

En caso de no tener cargado el dataframe, se lee el csv

In [ ]:
df = pd.read_csv('bookshop_1.csv')

Se agrega un buscador de palabras para título y se guarda en un array matches_titulo con los nombres de los títulos que contienen esa palabra. 
Como el dataframe está en minúscula, la palabra buscada se transforma a lowercase.

In [76]:
links_list_titulo = [*set(book_info['titulo'])]

palabra_clave_titulo = input("Palabra buscada en titulo: ")
# ejemplo: castigo
matches_titulo = []
for match_titulo in links_list_titulo:
    if palabra_clave_titulo.lower() in str(match_titulo):
        matches_titulo.append(match_titulo)
print(matches_titulo)
print(len(matches_titulo))


['crimen y castigo', 'castigos justificados (serie bergman 5)']
2


Se agrega un buscador de palabras para descripción y se guarda en un dataframe matches_descripcion con todos los datos de los ejemplares que contienen esa palabra en su descripción. Como el dataframe está en minúscula, la palabra buscada se transforma a lowercase.

In [78]:
links_list_descripcion = [*set(book_info['descripcion'])]

palabra_clave_descripcion = input("Palabra buscada en descripcion: ")
# ejemplo: amor
matches_descripcion = pd.DataFrame()
for i in range(len(book_info)):
    if palabra_clave_descripcion.lower() in str(book_info.loc[i]['descripcion']):
        matches_descripcion = pd.concat([matches_descripcion, book_info.loc[i:i]])
display(matches_descripcion.head())
print(matches_descripcion.shape)

,titulo,link,precio,disponibilidad,ISBN,autor,editorial,edicion,idioma,paginas,tapa,descripcion,fecha
3,sueño en el pabellón rojo ii,https://www.bookshop.com.uy/sue-o-en-el-pabell...,1350,disponible,9788481098358,cao xueqin,galaxia gutenberg,2023,español,1195,rústica,segundo volumen del gran clásico de la literat...,2023-12-13
6,cuentos de amor de locura y de muerte,https://www.bookshop.com.uy/cuentos-de-amor-de...,530,disponible,9789877253528,horacio quiroga,de bolsillo,2023,español,160,rústica,«pasa algo muy especial con los cuentos de qui...,2023-12-13
7,sueño en el pabellón rojo,https://www.bookshop.com.uy/sue-o-en-el-pabell...,1390,disponible,9788418526800,cao xueqin,océano,2023,español,220,rústica,sueño en el pabellón rojo es el gran clásico d...,2023-12-13
14,madame bovary,https://www.bookshop.com.uy/madame-bovary.html,690,disponible,9788491056294,gustave flaubert,penguin clásicos ‐ de bolsillo,2023,español,352,tapa dura,emma rouault es una joven de origen campesino ...,2023-12-13
21,pinocho,https://www.bookshop.com.uy/pinocho.html,590,disponible,9788417127596,"collodi, carlo",oceano,NaN,español,256,NaN,pinocho no es solo un libro infantil que perma...,2023-12-13


(313, 13)


Se agrega un buscador de ejemplares por precio, estableciendo un precio mínimo y un precio máximo y se guarda en un dataframe matches_precio con todos los ejemplares que cumplan con esa condición. En caso de querer buscar un único precio, se debe escribir el mismo número en los dos inputs

In [82]:
precio_minimo = input("Precio minimo: ")
precio_maximo = input("Precio maximo: ")
# ejemplo: precio_minimo = 500, precio_maximo = 1000
matches_precio = pd.DataFrame()
for i in range(len(df)):
    if (df.loc[i]['precio'] >= int(precio_minimo)) and (df.loc[i]['precio'] <= int(precio_maximo)):
        matches_precio = pd.concat([matches_precio, df.loc[i:i]])
display(matches_precio.head())
print(matches_precio.shape)

,titulo,link,precio,disponibilidad,ISBN,autor,editorial,edicion,idioma,paginas,tapa,descripcion,fecha
0,pride and prejudice,https://www.bookshop.com.uy/pride-and-prejudic...,790,disponible,9781840227932,jane austen,9781840227932,2023,español,160,tapa dura,pride and prejudice by jane austen. austen's m...,2023-12-13
1,novela de ajedrez,https://www.bookshop.com.uy/novela-de-ajedrez....,890,disponible,9788418933523,stefan zweig,américa latina,2023,español,96,tapa dura,un crucero que cubre la ruta de nueva york a b...,2023-12-13
4,orgullo y prejuicio,https://www.bookshop.com.uy/orgullo-y-prejuici...,995,disponible,9786071436092,jane austen,mirlo,2021,español,480,tapa dura,jane austen una de las autoras más reconocidas...,2023-12-13
5,1984 - rebelión en la granja,https://www.bookshop.com.uy/1984-rebelion-en-l...,960,disponible,9788497945592,george orwell,edimat,2023,español,352,tapa dura,"la novela 1984 es un ejemplo, quizá el más con...",2023-12-13
6,cuentos de amor de locura y de muerte,https://www.bookshop.com.uy/cuentos-de-amor-de...,530,disponible,9789877253528,horacio quiroga,de bolsillo,2023,español,160,rústica,«pasa algo muy especial con los cuentos de qui...,2023-12-13


(1596, 13)
